In [ ]:
from queue import PriorityQueue

# Node Class
class Node:
    def __init__(self, position, parent=None):
        self.position = position    # (row, col) coordinate in the maze
        self.parent = parent        # Pointer to parent node (used to reconstruct path)
        self.h = 0                  # Heuristic value (estimated cost to goal)
        self.f = 0                  # Final evaluation value (used for priority)

    def __lt__(self, other):        # This tells PriorityQueue how to compare two nodes. Will prioritize the node with smaller f value.
        return self.f < other.f


# Manhattan Distance Heuristic
def heuristic(current_pos, goal_pos):
    return abs(current_pos[0] - goal_pos[0]) + abs(current_pos[1] - goal_pos[1])


# Best First Search to One Goal
def best_first_search_single_goal(maze, start, goal):
    rows, cols = len(maze), len(maze[0])    # Get maze dimensions

    start_node = Node(start)                # Create starting node
    frontier = PriorityQueue()              # Create priority queue (frontier)
    frontier.put(start_node)                # Put starting node inside frontier

    visited = set()                         # Set to keep track of visited positions

    while not frontier.empty():
        current_node = frontier.get()       # Remove node with smallest f value
        current_pos = current_node.position

        if current_pos == goal:             # If we reached the goal, reconstruct path
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]

        visited.add(current_pos)            # Mark current position as visited

        for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:               # Possible travel movement, down, up, right, left
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)    # Move to new position

            if (0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols and maze[new_pos[0]][new_pos[1]] != 1 and new_pos not in visited):
                new_node = Node(new_pos, current_node)              # Create New Node
                new_node.h = heuristic(new_pos, goal)               # Calculate heuristic (distance to goal)
                new_node.f = new_node.h                             # f(n) = h(n) (Greedy Best First)
                frontier.put(new_node)                              # Add node to frontier
    return None                                                     # If no path found

# Enhanced Multi-Goal Search
def best_first_search_multiple_goals(maze, start, goals):
    current_start = start
    total_path = []
    remaining_goals = goals.copy()      # Copy all the goals that have to be reached

    while remaining_goals:
        nearest_goal = min( remaining_goals, key=lambda g: heuristic(current_start, g))     # Choose nearest goal using heuristic

        path = best_first_search_single_goal(maze, current_start, nearest_goal)

        if path is None:
            print("One of the goals is unreachable!")
            return None

        # Avoid duplicating start node in path
        if total_path:
            total_path.extend(path[1:])
        else:
            total_path.extend(path)

        current_start = nearest_goal
        remaining_goals.remove(nearest_goal)

    return total_path


# Example Maze with Multiple Goals
maze = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0]
]

start = (0, 0)
goals = [(4, 4), (2, 3)]   # Multiple goals

print("Enhanced Best First Search (Multiple Goals)")
path = best_first_search_multiple_goals(maze, start, goals)

if path:
    print("Path covering all goals:")
    print(path)
    print("Total Steps:", len(path)-1)
else:
    print("No path found.")


Enhanced Best First Search (Multiple Goals)
Path covering all goals:
[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 4), (4, 4)]
Total Steps: 8


In [1]:
import random

class Node:
    def __init__(self, position):
        self.position = position
        self.parent = None
        self.g = float('inf')
        self.h = 0
        self.f = float('inf')

    def __lt__(self, other):
        return self.f < other.f

# Heuristic (Manhattan)
def heuristic(a, b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

def dynamic_cost():
    return random.randint(1, 5)

def dynamic_a_star(maze, start, goal):

    rows, cols = len(maze), len(maze[0])

    nodes = {}
    open_list = []
    closed = set()

    start_node = Node(start)
    start_node.g = 0
    start_node.h = heuristic(start, goal)
    start_node.f = start_node.h

    nodes[start] = start_node
    open_list.append(start_node)

    while open_list:

        open_list.sort()
        current = open_list.pop(0)

        if current.position == goal:
            path = []
            while current:
                path.append(current.position)
                current = current.parent
            return path[::-1]

        closed.add(current.position)

        for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
            new_pos = (current.position[0]+dx, current.position[1]+dy)

            if (0 <= new_pos[0] < rows and
                0 <= new_pos[1] < cols and
                maze[new_pos[0]][new_pos[1]] != 1):

                cost = dynamic_cost()  # cost changes dynamically

                if new_pos not in nodes:
                    nodes[new_pos] = Node(new_pos)

                neighbor = nodes[new_pos]

                if new_pos in closed:
                    continue

                new_g = current.g + cost

                if new_g < neighbor.g:
                    neighbor.g = new_g
                    neighbor.h = heuristic(new_pos, goal)
                    neighbor.f = neighbor.g + neighbor.h
                    neighbor.parent = current

                    if neighbor not in open_list:
                        open_list.append(neighbor)

    return None


# Main Section
maze = [
    [0,0,0,1,1],
    [0,1,0,1,0],
    [0,0,1,0,1],
    [0,0,0,0,1],
    [0,0,0,1,0],
]

start = (0,0)
goal = (2,3)

print("Dynamic A* Search Running...")
path = dynamic_a_star(maze, start, goal)

if path:
    print("Optimal Path Found:")
    print(path)
else:
    print("No Path Found.")


Dynamic A* Search Running...
Optimal Path Found:
[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3)]


In [33]:
from queue import PriorityQueue

# Define the graph (each node can have at most 2 children)
graph = {
    'A': [('B', 3), ('C', 2)],
    'B': [('D', 4), ('E', 6)],
    'C': [('F', 5), ('G', 3)],
    'D': [('H', 2)],
    'E': [('I', 3)],
    'F': [('J', 4)],
    'G': [],
    'H': [],
    'I': [],
    'J': []
}

# Heuristic values for each node (to nearest delivery node)
heuristics = {
    'A': 7, 'B': 5, 'C': 6, 'D': 2, 'E': 9,
    'F': 8, 'G': 10, 'H': 3, 'I': 11, 'J': 12
}

# Delivery nodes and their deadlines
deliveries = {
    'B': 5,  # Should succeed
    'D': 6,  # Should fail (arrival time will exceed)
    'H': 11,   # Should fail (tight path)
    'I': 8   # Should fail (tight path)
}

def greedy_best_first_search(start, graph, heuristics, deliveries):
    # Sort delivery nodes by deadline (earliest first)
    sorted_deliveries = sorted(deliveries.items(), key=lambda x: x[1])
    
    current_node = start
    total_time = 0
    
    print("Starting deliveries from node:", start)
    
    for goal, deadline in sorted_deliveries:
        print(f"\nNext delivery node: {goal} (Deadline: {deadline})")
        
        # Priority Queue for GBFS
        frontier = PriorityQueue()
        frontier.put((heuristics[current_node], [current_node]))
        visited = set()
        path_found = False
        
        while not frontier.empty():
            _, path = frontier.get()
            node = path[-1]
            
            if node == goal:
                path_found = True
                # Add path cost (sum of edge weights)
                for i in range(len(path)-1):
                    for neighbor, cost in graph[path[i]]:
                        if neighbor == path[i+1]:
                            total_time += cost
                print(f"Path taken: {' -> '.join(path)}, Time spent: {total_time}")
                
                # Check delivery success
                if total_time <= deadline:
                    print(f"Delivery to {goal} SUCCESSFUL")
                else:
                    print(f"Delivery to {goal} FAILED (Exceeded deadline)")
                
                # Move current position to this node
                current_node = goal
                break
            
            if node not in visited:
                visited.add(node)
                for neighbor, _ in graph[node]:
                    if neighbor not in visited:
                        # Greedy: priority = heuristic value
                        frontier.put((heuristics[neighbor], path + [neighbor]))
        
        if not path_found:
            print(f"Delivery to {goal} FAILED (No path found)")

# Run the simulation
greedy_best_first_search('A', graph, heuristics, deliveries)

Starting deliveries from node: A

Next delivery node: B (Deadline: 5)
Path taken: A -> B, Time spent: 3
Delivery to B SUCCESSFUL

Next delivery node: D (Deadline: 6)
Path taken: B -> D, Time spent: 7
Delivery to D FAILED (Exceeded deadline)

Next delivery node: I (Deadline: 8)
Delivery to I FAILED (No path found)

Next delivery node: H (Deadline: 11)
Path taken: D -> H, Time spent: 9
Delivery to H SUCCESSFUL
